---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
from google.colab import drive
drive.mount('/data')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /data


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

!pip install xlrd

    100% |████████████████████████████████| 112kB 4.2MB/s 


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [0]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [186]:
def get_list_of_university_towns():
  df = pd.read_csv('/data/My Drive/university_towns.txt', sep="\n", header=None, names=["university"])
#   df['university'] = df['university'].str.split('[').str[0]
  df['university'] = df['university'].str.split('(').str[0].str.rstrip()
  data = pd.DataFrame([],columns=['State','RegionName'])
  df['compare'] = df['university'].str.contains('[edit]', regex=False)
  df['university'] = df['university'].str.split('[').str[0]
  s =''
  for index, row in df.iterrows():
    if row['compare']:
      s = row['university']
    else:
      df2 = pd.DataFrame([[s, row['university']]], columns=['State','RegionName'])
      data = data.append(df2, ignore_index=True)
  
  return data
get_list_of_university_towns()


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [27]:
def  get_data_gdp():
  gdp = pd.read_excel('/data/My Drive/gdplev.xls', skiprows=range(219))
  gdp.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 7'], inplace=True)
  gdp.columns = ['time', 'gdp', 'chained2009']
  return gdp
get_data_gdp()

,time,gdp,chained2009
0,2000q1,10031.0,12359.1
1,2000q2,10278.3,12592.5
2,2000q3,10357.4,12607.7
3,2000q4,10472.3,12679.3
4,2001q1,10508.1,12643.3
5,2001q2,10638.4,12710.3
6,2001q3,10639.5,12670.1
7,2001q4,10701.3,12705.3
8,2002q1,10834.4,12822.3
9,2002q2,10934.8,12893.0


In [61]:
def get_recession_start():
  gdp = get_data_gdp()
  
  counter = 0
  for i in range(len(gdp)):
    if(counter!=2):
      if(gdp['gdp'][i]> gdp['gdp'][i+1]):
        counter= counter+1
    else:
      break
  return gdp['time'][i]
get_recession_start()

'2008q4'

In [62]:
def get_recession_end():
  gdp = get_data_gdp()
  start = get_recession_start()
  start_index = gdp[gdp['time'] == start].index.tolist()[0]
  gdp=gdp.iloc[start_index:]
  counter = 0
  for i in range(len(gdp)):
    if(counter!=2):
        if(gdp.iloc[i][1]< gdp.iloc[i+1][1]):
            counter= counter+1
    else:
        break
  
  return gdp.iloc[i][0]
       
get_recession_end()

'2009q4'

In [63]:
def get_recession_bottom():
  gdp = get_data_gdp()
  
  start = get_recession_start()
  s_index = gdp[gdp.time == start].index[0]
  ends = get_recession_end()
  e_index = gdp[gdp.time == ends].index[0]

  index = s_index
  mins = ""
  while True:
    if gdp.iloc[index+1].gdp < gdp.iloc[index].gdp:
      mins = gdp.iloc[index+1].time
      mins2 = index + 1
    else:
      pass
      index += 1
      if index >= e_index: break

    return mins
get_recession_bottom()

'2009q1'

In [84]:
def convert_housing_data_to_quarters():
  house = pd.read_csv("/data/My Drive/City_Zhvi_AllHomes.csv")
  ihouse = house.loc[:,"2000-01":"2016-08"]
  ihouse.columns = pd.to_datetime(ihouse.columns).to_period(freq="M")
  ghouse = ihouse.groupby(ihouse.columns.asfreq("q"),axis=1).sum()
  house = (pd.merge(house.loc[:,"RegionID":"SizeRank"],ghouse,left_index=True,right_index=True,how="inner")
  .set_index(["State","RegionName"]).iloc[:,4:71])
  return house
convert_housing_data_to_quarters()


,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1546400.0,1568400.0,1584200.0,1596800.0,1622400.0,1671600.0,1718500.0,1748600.0,1774900.0,1174400.0
CA,Los Angeles,621200.0,643400.0,662900.0,678500.0,699000.0,717300.0,735200.0,759100.0,785900.0,818100.0,...,1494100.0,1527200.0,1556600.0,1586400.0,1614500.0,1641800.0,1673200.0,1698100.0,1732400.0,1168100.0
IL,Chicago,415200.0,430900.0,443600.0,456400.0,470800.0,485400.0,499200.0,511300.0,526500.0,532700.0,...,577900.0,587300.0,603800.0,603200.0,618100.0,624900.0,623700.0,618200.0,624600.0,424000.0
PA,Philadelphia,159000.0,160900.0,162400.0,164100.0,166000.0,166600.0,168800.0,172600.0,177400.0,182200.0,...,341200.0,345900.0,347000.0,348600.0,353900.0,363700.0,366600.0,370300.0,380800.0,257400.0
AZ,Phoenix,335500.0,343100.0,348000.0,352200.0,358800.0,364700.0,368100.0,372900.0,379600.0,385100.0,...,492800.0,496100.0,505500.0,514600.0,522500.0,537200.0,551500.0,563700.0,574300.0,390400.0
NV,Las Vegas,397800.0,403100.0,406200.0,411000.0,418600.0,425200.0,430100.0,438400.0,448000.0,452800.0,...,510200.0,520200.0,526400.0,532500.0,544800.0,560300.0,571900.0,583800.0,591600.0,399900.0
CA,San Diego,668700.0,703100.0,736300.0,768100.0,801600.0,828800.0,853500.0,875800.0,903700.0,938600.0,...,1440600.0,1467100.0,1489300.0,1510100.0,1536200.0,1559300.0,1576400.0,1588000.0,1608700.0,1079500.0
TX,Dallas,253400.0,251600.0,254600.0,263500.0,269200.0,267900.0,267200.0,272700.0,277700.0,281400.0,...,319900.0,326700.0,334600.0,341100.0,363400.0,385700.0,403800.0,421500.0,433800.0,298600.0
CA,San Jose,1122800.0,1219700.0,1295600.0,1366500.0,1412000.0,1410600.0,1370400.0,1336700.0,1324300.0,1373300.0,...,2038200.0,2091100.0,2144800.0,2194300.0,2270200.0,2329200.0,2367400.0,2410800.0,2456800.0,1644400.0


In [193]:
def run_ttest():
  data_quarters = convert_housing_data_to_quarters()
  data_quarters.columns = data_quarters.columns.astype(str).str.lower()
  data_quarters.reset_index()
  start = get_recession_start()
  bottom = get_recession_bottom()
  quarters = data_quarters.columns
  start_idx = list(quarters).index(start)
  before = quarters[:start_idx][-1]
  new_data = data_quarters.loc[:, [before, bottom]].reset_index()
  new_data = new_data.assign(price_ratio = new_data[before] / new_data[bottom])
  
  uni_town = get_list_of_university_towns()['RegionName'].values
  uni_town = set(uni_town)
  
  new_data = new_data.assign(is_university_town = [True if x in uni_town else False for x in new_data['RegionName'].values])
  
  university_towns = new_data[new_data['is_university_town'] == True]['price_ratio'].values
  non_university_towns = new_data[new_data['is_university_town'] == False]['price_ratio'].values
  

  ttest_res = ttest_ind(university_towns, non_university_towns, nan_policy='omit')
  reject_null_hypothesis = None
  lower_mean_price_ratio = None
  if ttest_res[1] < 0.01:
    reject_null_hypothesis = True
  else:
    reject_null_hypothesis = False
  if ttest_res[0] < 0:
    lower_mean_price_ratio = "university town"
  else:
    lower_mean_price_ratio = "non-university town"
  return reject_null_hypothesis, ttest_res[1], lower_mean_price_ratio



run_ttest()

(True, 0.0008981682297064972, 'university town')